In [ ]:
!pip install segmentation-models-3D

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.0 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 


In [ ]:
import tensorflow as tf
import numpy as np
import keras.backend as K
import segmentation_models_3D as sm

from matplotlib import pyplot as plt
from IPython import display
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model
from keras.metrics import MeanIoU

Segmentation Models: using `tf.keras` framework.


Data

In [ ]:
# get data files
train_x = np.load('/content/drive/MyDrive/framebot_V2/data/train_input.npy', allow_pickle=True)
train_y = np.load('/content/drive/MyDrive/framebot_V2/data/train_label.npy', allow_pickle=True)

val_x = np.load('/content/drive/MyDrive/framebot_V2/data/val_input.npy', allow_pickle=True)
val_y = np.load('/content/drive/MyDrive/framebot_V2/data/val_label.npy', allow_pickle=True)

In [ ]:
# add channel for input
train_x = train_x.reshape(train_x.shape[0], 32, 32, 32, 1)
val_x = val_x.reshape(val_x.shape[0], 32, 32, 32, 1)

In [ ]:
# move axis for labels
train_y = np.moveaxis(train_y, 1, -1)
val_y = np.moveaxis(val_y, 1, -1)

In [ ]:
# Hyper parameters
AUTO = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 8
BUFFER_SIZE = 12600

STEPS_PER_EPOCH = len(train_x) // BATCH_SIZE
VALID_STEPS = len(val_x) // BATCH_SIZE

train_steps = len(train_x) // BATCH_SIZE

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

In [ ]:
train_dataset.element_spec

(TensorSpec(shape=(None, 32, 32, 32, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 32, 32, 32, 7), dtype=tf.float32, name=None))

In [ ]:
val_dataset.element_spec

(TensorSpec(shape=(None, 32, 32, 32, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 32, 32, 32, 7), dtype=tf.float32, name=None))

Model

In [ ]:
def get_model():
    inputs = tf.keras.layers.Input((32, 32, 32, 1))
    output_channels = 7


    # Entry block 
    c1 = layers.Conv3D(32, 3, kernel_initializer='he_normal', padding="same")(inputs)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Activation("relu")(c1)
    c1 = layers.Dropout(0.1)(c1)

    c1 = layers.Conv3D(32, 3, kernel_initializer='he_normal', padding="same")(c1)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Activation("relu")(c1)
    c1 = layers.Dropout(0.1)(c1)

    c1 = layers.Conv3D(32, 3, kernel_initializer='he_normal', padding="same")(c1)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Activation("relu")(c1)
    c1 = layers.Dropout(0.1)(c1)

    c1 = layers.Conv3D(32, 3, kernel_initializer='he_normal', padding="same")(c1)
    c1 = layers.BatchNormalization()(c1)
    c1 = layers.Activation("relu")(c1)

    p1 = layers.MaxPooling3D(2, strides=2, padding="same")(c1)


    # Downsample block 1
    c2 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(p1)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Activation("relu")(c2)
    c2 = layers.Dropout(0.1)(c2)

    c2 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(c2)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Activation("relu")(c2)
    c2 = layers.Dropout(0.1)(c2)

    c2 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(c2)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Activation("relu")(c2)
    c2 = layers.Dropout(0.1)(c2)

    c2 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(c2)
    c2 = layers.BatchNormalization()(c2)
    c2 = layers.Activation("relu")(c2)

    p2 = layers.MaxPooling3D(2, strides=2, padding="same")(c2)


    # Downsample block 2
    c3 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(p2)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Activation("relu")(c3)
    c3 = layers.Dropout(0.2)(c3)

    c3 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(c3)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Activation("relu")(c3)
    c3 = layers.Dropout(0.2)(c3)

    c3 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(c3)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Activation("relu")(c3)
    c3 = layers.Dropout(0.2)(c3)

    c3 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(c3)
    c3 = layers.BatchNormalization()(c3)
    c3 = layers.Activation("relu")(c3)

    p3 = layers.MaxPooling3D(2, strides=2, padding="same")(c3)
    

    # Downsample block 3
    c4 = layers.Conv3D(256, 3, kernel_initializer='he_normal', padding="same")(p3)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Activation("relu")(c4)
    c4 = layers.Dropout(0.2)(c4)

    c4 = layers.Conv3D(256, 3, kernel_initializer='he_normal', padding="same")(c4)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Activation("relu")(c4)
    c4 = layers.Dropout(0.2)(c4)

    c4 = layers.Conv3D(256, 3, kernel_initializer='he_normal', padding="same")(c4)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Activation("relu")(c4)
    c4 = layers.Dropout(0.2)(c4)

    c4 = layers.Conv3D(256, 3, kernel_initializer='he_normal', padding="same")(c4)
    c4 = layers.BatchNormalization()(c4)
    c4 = layers.Activation("relu")(c4)

    p4 = layers.MaxPooling3D(2, strides=2, padding="same")(c4)


    # Downsample block 4
    c5 = layers.Conv3D(512, 3, kernel_initializer='he_normal', padding="same")(p4)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Activation("relu")(c5)
    c5 = layers.Dropout(0.2)(c5)

    c5 = layers.Conv3D(512, 3, kernel_initializer='he_normal', padding="same")(c5)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Activation("relu")(c5)
    c5 = layers.Dropout(0.2)(c5)

    c5 = layers.Conv3D(512, 3, kernel_initializer='he_normal', padding="same")(c5)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Activation("relu")(c5)
    c5 = layers.Dropout(0.2)(c5)

    c5 = layers.Conv3D(512, 3, kernel_initializer='he_normal', padding="same")(c5)
    c5 = layers.BatchNormalization()(c5)
    c5 = layers.Activation("relu")(c5)

    p5 = layers.MaxPooling3D(2, strides=2, padding="same")(c5)


    # Bottom block
    c6 = layers.Conv3D(1024, 3, kernel_initializer='he_normal', padding="same")(p5)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Activation("relu")(c6)
    c6 = layers.Dropout(0.3)(c6)

    c6 = layers.Conv3D(2048, 3, kernel_initializer='he_normal', padding="same")(c6)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Activation("relu")(c6)
    c6 = layers.Dropout(0.3)(c6)

    c6 = layers.Conv3D(2048, 3, kernel_initializer='he_normal', padding="same")(c6)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Activation("relu")(c6)
    c6 = layers.Dropout(0.3)(c6)

    c6 = layers.Conv3D(1024, 3, kernel_initializer='he_normal', padding="same")(c6)
    c6 = layers.BatchNormalization()(c6)
    c6 = layers.Activation("relu")(c6)


    # Upsample block 1
    u7 = tf.keras.layers.Conv3DTranspose(512, 2, strides=2, padding="same")(c6)
    u7 = layers.Concatenate()([u7, c5])

    c7 = layers.Conv3D(512, 3, kernel_initializer='he_normal', padding="same")(u7)
    c7 = layers.BatchNormalization()(c7)
    c7 = layers.Activation("relu")(c7)
    c7 = layers.Dropout(0.2)(c7)

    c7 = layers.Conv3D(512, 3, kernel_initializer='he_normal', padding="same")(c7)
    c7 = layers.BatchNormalization()(c7)
    c7 = layers.Activation("relu")(c7)
    c7 = layers.Dropout(0.2)(c7)

    c7 = layers.Conv3D(512, 3, kernel_initializer='he_normal', padding="same")(c7)
    c7 = layers.BatchNormalization()(c7)
    c7 = layers.Activation("relu")(c7)
    c7 = layers.Dropout(0.2)(c7)

    c7 = layers.Conv3D(512, 3, kernel_initializer='he_normal', padding="same")(c7)
    c7 = layers.BatchNormalization()(c7)
    c7 = layers.Activation("relu")(c7)

    c7 = layers.Conv3D(512, 3, kernel_initializer='he_normal', padding="same")(c7)


    # Upsample block 2
    u8 = tf.keras.layers.Conv3DTranspose(256, 2, strides=2, padding="same")(c7)
    u8 = layers.Concatenate()([u8, c4])

    c8 = layers.Conv3D(256, 3, kernel_initializer='he_normal', padding="same")(u8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Activation("relu")(c8)
    c8 = layers.Dropout(0.2)(c8)

    c8 = layers.Conv3D(256, 3, kernel_initializer='he_normal', padding="same")(c8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Activation("relu")(c8)
    c8 = layers.Dropout(0.2)(c8)

    c8 = layers.Conv3D(256, 3, kernel_initializer='he_normal', padding="same")(c8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Activation("relu")(c8)
    c8 = layers.Dropout(0.2)(c8)

    c8 = layers.Conv3D(256, 3, kernel_initializer='he_normal', padding="same")(c8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Activation("relu")(c8)

    c8 = layers.Conv3D(256, 3, kernel_initializer='he_normal', padding="same")(c8)


    # Upsample block 3
    u9 = tf.keras.layers.Conv3DTranspose(128, 2, strides=2, padding="same")(c8)
    u9 = layers.Concatenate()([u9, c3])

    c9 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(u9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Activation("relu")(c9)
    c9 = layers.Dropout(0.2)(c9)

    c9 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(c9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Activation("relu")(c9)
    c9 = layers.Dropout(0.2)(c9)

    c9 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(c9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Activation("relu")(c9)
    c9 = layers.Dropout(0.2)(c9)

    c9 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(c9)


    # Upsample block 4
    u8 = tf.keras.layers.Conv3DTranspose(64, 2, strides=2, padding="same")(c7)
    u8 = layers.Concatenate()([u8, c4])

    c8 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(u8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Activation("relu")(c8)
    c8 = layers.Dropout(0.2)(c8)

    c8 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(c8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Activation("relu")(c8)
    c8 = layers.Dropout(0.2)(c8)

    c8 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(c8)
    c8 = layers.BatchNormalization()(c8)
    c8 = layers.Activation("relu")(c8)
    c8 = layers.Dropout(0.2)(c8)

    c8 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(c8)
    

    # Upsample block 5
    u9 = tf.keras.layers.Conv3DTranspose(128, 2, strides=2, padding="same")(c8)
    u9 = layers.Concatenate()([u9, c3])

    c9 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(u9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Activation("relu")(c9)
    c9 = layers.Dropout(0.2)(c9)

    c9 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(c9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Activation("relu")(c9)
    c9 = layers.Dropout(0.2)(c9)

    c9 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(c9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Activation("relu")(c9)
    c9 = layers.Dropout(0.2)(c9)

    c9 = layers.Conv3D(128, 3, kernel_initializer='he_normal', padding="same")(c9)
    c9 = layers.BatchNormalization()(c9)
    c9 = layers.Activation("relu")(c9)


    # Upsample block 6
    u10 = tf.keras.layers.Conv3DTranspose(64, 2, strides=2, padding="same")(c9)
    u10 = layers.Concatenate()([u10, c2])

    c10 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(u10)
    c10 = layers.BatchNormalization()(c10)
    c10 = layers.Activation("relu")(c10)
    c10 = layers.Dropout(0.2)(c10)

    c10 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(c10)
    c10 = layers.BatchNormalization()(c10)
    c10 = layers.Activation("relu")(c10)
    c10 = layers.Dropout(0.2)(c10)

    c10 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(c10)
    c10 = layers.BatchNormalization()(c10)
    c10 = layers.Activation("relu")(c10)
    c10 = layers.Dropout(0.2)(c10)

    c10 = layers.Conv3D(64, 3, kernel_initializer='he_normal', padding="same")(c10)
    c10 = layers.BatchNormalization()(c10)
    c10 = layers.Activation("relu")(c10)


    # Upsample block 7
    u11 = tf.keras.layers.Conv3DTranspose(32, 2, strides=2, padding="same")(c10)
    u11 = layers.Concatenate()([u11, c1])

    c11 = layers.Conv3D(32, 3, kernel_initializer='he_normal', padding="same")(u11)
    c11 = layers.BatchNormalization()(c11)
    c11 = layers.Activation("relu")(c11)
    c11 = layers.Dropout(0.2)(c11)

    c11 = layers.Conv3D(32, 3, kernel_initializer='he_normal', padding="same")(c11)
    c11 = layers.BatchNormalization()(c11)
    c11 = layers.Activation("relu")(c11)
    c11 = layers.Dropout(0.2)(c11)

    c11 = layers.Conv3D(32, 3, kernel_initializer='he_normal', padding="same")(c11)
    c11 = layers.BatchNormalization()(c11)
    c11 = layers.Activation("relu")(c11)
    c11 = layers.Dropout(0.2)(c11)

    c11 = layers.Conv3D(32, 3, kernel_initializer='he_normal', padding="same")(c11)
    c11 = layers.BatchNormalization()(c11)
    c11 = layers.Activation("relu")(c11)
    

    # Add per-pixel classification layer
    outputs = layers.Conv3D(output_channels, 1, activation="softmax")(c11)


    # Define the model
    model = keras.Model(inputs, outputs)
    return model

In [ ]:
tf.keras.backend.clear_session()
# model = get_model()
model = get_model()
# tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 32,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d (Conv3D)                (None, 32, 32, 32,   896         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 32,   128        ['conv3d[0][0]']                 
 alization)                     32)                                                           

Train model

In [ ]:
# model location
# small model
save_model_path = "/content/drive/MyDrive/framebot_V2/model/framebot_v2.0_3d_Unet_1.2.h5"

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/framebot_V2/checkpoints/'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_iou_score',
    mode='max',
    save_best_only=True)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [ ]:
dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

loss = sm.losses.categorical_focal_dice_loss

metrics = [sm.metrics.IOUScore(threshold=0.5)]
# metrics = ['accuracy']

In [ ]:
# Load model for retraining
# model = load_model(save_model_path, custom_objects={'focal_loss': focal_loss})

# Load model for testing - ignore loss error
# model = load_model(save_model_path, compile=False)

In [ ]:
# Compile model

###HIDE THIS IF LOADING MODEL FOR TESTING OR USE

model.compile(
    loss = loss,
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=metrics
    )


In [ ]:
EPOCHS = 1000

In [ ]:
history = model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs = EPOCHS,
    shuffle = True,
    verbose = 2,
    callbacks=[model_checkpoint_callback, early_stopping]
)

In [ ]:
# save model
model.save(save_model_path)

In [ ]:
###
#plot the training and validation IoU and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['iou_score']
val_acc = history.history['val_iou_score']

plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and validation IOU')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

# Evaluate Model

In [ ]:
test_x = np.load('/content/drive/MyDrive/framebot_V2/data/test_input.npy', allow_pickle=True)
test_y = np.load('/content/drive/MyDrive/framebot_V2/data/test_label.npy', allow_pickle=True)

In [ ]:
# add channel for input
test_x = test_x.reshape(test_x.shape[0], 32, 32, 32, 1)

# move axis for labels
test_y = np.moveaxis(test_y, 1, -1)

In [ ]:
# Hyper parameters
AUTO = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 8
BUFFER_SIZE = 12600

STEPS_PER_EPOCH = len(test_x) // BATCH_SIZE
VALID_STEPS = len(test_x) // BATCH_SIZE

train_steps = len(test_x) // BATCH_SIZE

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_dataset = test_dataset.shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
test_dataset.element_spec

(TensorSpec(shape=(None, 32, 32, 32, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 32, 32, 32, 7), dtype=tf.float32, name=None))

In [ ]:
model.evaluate(test_dataset)

125/125 [==============================] - 26s 87ms/step - loss: 0.0711 - iou_score: 0.8893


[0.07107943296432495, 0.8893222212791443]